# Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install natasha

     |████████████████████████████████| 34.4 MB 149 kB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 41 kB 120 kB/s 
     |████████████████████████████████| 49 kB 5.6 MB/s 
     |████████████████████████████████| 8.2 MB 29.8 MB/s 
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26119 sha256=f4b1804a613d3c154a2085a569b400b82663e1ecabfe1077176d1f44797c92e7
  Stored in directory: /root/.cache/pip/wheels/16/85/bd/1001cbb46dcfb71c2001cd7401c6fb250392f22a81ce3722f7
Successfully built intervaltree
  Attempting uninstall: intervaltree
    Found existing installation: intervaltree 2.1.0
    Uninstalling intervaltree-2.1.0:
      Successfully uninstalled intervaltree-2.1.0


In [45]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm.notebook import tqdm

from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import warnings
warnings.filterwarnings('ignore')
device = "cuda" if torch.cuda.is_available() else "cpu"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data analysis

In [4]:
df = pd.read_csv('/content/drive/MyDrive/skb_kontur/train.tsv', sep='\t')

Для начала посмотрим на датасет

In [5]:
df.sample(10)

,title,is_fake
5426,Star Trek Online стартовал с миллионом пользов...,0
1758,Lenovo добавит ноутбукам новых ядер,0
3697,Юрий Лоза раскритиковал качество снимков с кос...,1
3016,Банки-кредиторы «Трансаэро» рассмотрят сценари...,0
5204,Ветеринара обвинили в пропаганде АУЕ из-за пос...,1
5718,В сентябре в Москве стартуют гонки на «Лафетах»,1
4890,Корпорация ТВЭЛ и Энергоатом договорились о по...,0
3611,Священник рассказал о молитвах российских олим...,0
1415,Итоги года. Наука,1
3178,Михалков снимет российский аналог Капитана Аме...,1


In [6]:
df.shape

(5758, 2)

In [7]:
df.dtypes

title      object
is_fake     int64
dtype: object

In [8]:
df['is_fake'].unique()

array([1, 0])

In [9]:
df.count()

title      5758
is_fake    5758
dtype: int64

In [10]:
df.isnull().sum()

title      0
is_fake    0
dtype: int64

Выбросов нет, все строки должны иметь правильную разметку

In [11]:
df['is_fake'].value_counts()

1    2879
0    2879
Name: is_fake, dtype: int64

Классы сбалансированны

# Tokenization
Токенизируем наши данные при помощи библиотеки Natasha.

In [28]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

Возьмем стоп-слова из библиотеки nltk

In [13]:
stopwords_ru = stopwords.words("russian")

Розабьем заголовки новостей на слова, применим к ним лемматизацию и удалим стоп-слова.

In [57]:
def get_tokens(df):
    tokens = []
    for text in df['title']:
        doc = Doc(text)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        for token in doc.tokens:
            token.lemmatize(morph_vocab)
        tok = [_.lemma for _ in doc.tokens if _.lemma not in stopwords_ru]
        tokens.append(tok)
    return tokens

на выходе получается list, заполненный токенами для каждого заголовка

In [58]:
tokens = get_tokens(df)
tokens[:3]

[['москвич',
  'владимир',
  'клутин',
  'прийти',
  'счет',
  'вмешательство',
  'американский',
  'выбор'],
 ['агент', 'кокорин', 'назвать', 'езда', 'встречка', 'житейский', 'история'],
 ['госдума',
  'рассмотреть',
  'возможность',
  'введение',
  'секретный',
  'стать',
  'уголовный',
  'кодекс']]

# Embedings
Воспользуемся готовыми эмбеддингами из библиотеки Natasha, обученными на новостях.

Представим каждый заголовок новостей в виде среднего арифметического эмбеддингов входящих в него слов.

In [60]:
def get_sent_emb(tokens):
    sent_emb = None
    for sentence in tokens:
        features = None # Лист, в котором будут храниться эмбеддинги слов текущего заголовка
        for word in sentence:
            if word in emb:
                if features is None:
                    features = [emb[word]]
                else:
                    features = np.concatenate((features, [emb[word]]), axis=0)
        # Нахождение среднего арифмитического всех эмбеддингов заголовка
        if sent_emb is None:
            sent_emb = [np.mean(features, axis=0)]
        else:
            sent_emb = np.concatenate((sent_emb, [np.mean(features, axis=0)]))
    return sent_emb

In [61]:
sent_emb = get_sent_emb(tokens)
sent_emb.shape

(5758, 300)

# Data Loders

In [17]:
X_train, X_test, y_train, y_test = train_test_split(sent_emb, df['is_fake'], test_size=0.1, random_state=42)

In [18]:
y_train.value_counts()

0    2607
1    2575
Name: is_fake, dtype: int64

Как мы видим, баланс классов остался на хорошем уровне

In [19]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [20]:
batch_size = 32
num_workers = 2

train_loader = DataLoader(list(zip(X_train, y_train)), batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=False)
valid_loader = DataLoader(list(zip(X_test, y_test)), batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False)

In [21]:
len(train_loader)

162

# Learning


In [23]:
vector_size = 300
num_classes = 2
num_epochs = 10

Напишем функцию обучения для наших моделей

In [22]:
def train_model(model, model_name):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.AdamW(model.parameters())

    loaders = {"train": train_loader, "valid": valid_loader}
    accuracy = {"train": [], "valid": []}
    best_accuracy = 0


    for epoch in tqdm(range(num_epochs)):
        for k, dataloader in loaders.items():
            epoch_correct = 0
            epoch_all = 0

            for x_batch, y_batch in dataloader:

                x_batch = x_batch.to(device)
                y_batch = y_batch.to(device)

                if k == "train":
                    model.train()
                    optimizer.zero_grad()
                    outp = model(x_batch)
                else:
                    model.eval()
                    with torch.no_grad():
                        outp = model(x_batch)

                preds = outp.argmax(-1)
                correct = (preds == y_batch).sum()
                all = len(outp)
                epoch_correct += correct.item()
                epoch_all += all

                if k == "train":
                    loss = criterion(outp, y_batch)
                    loss.backward()
                    optimizer.step()

            # if k == "train":
            #     print(f"Epoch: {epoch+1}")
            # print(f"Loader: {k}. Accuracy: {epoch_correct/epoch_all}")

            accuracy[k].append(epoch_correct/epoch_all)
            # Сохраним параметры модели в лучшей эпохе на диск
            if k == 'valid' and accuracy['valid'][-1] > best_accuracy:
                torch.save(model.state_dict(), f"/content/drive/MyDrive/skb_kontur/{model_name}.pt")
                best_accuracy = accuracy['valid'][-1]

    print('Best accuracy:\n')
    print('test:', round(max(accuracy['valid']), 5))
    print('train:', round(max(accuracy['train']), 5))
    # Загрузим в модель параметры, показавшие наилучший результат
    model.load_state_dict(torch.load(f"/content/drive/MyDrive/skb_kontur/{model_name}.pt", map_location=device))

Попробуем разные архитектуры полносвязных нейронных сетей

In [24]:
model1 = nn.Sequential(
    nn.BatchNorm1d(vector_size),
    nn.Linear(vector_size, 200),
    nn.BatchNorm1d(200),
    nn.ReLU(),
    nn.Linear(200, 100),
    nn.BatchNorm1d(100),
    nn.ReLU(),
    nn.Linear(100, 50),
    nn.BatchNorm1d(50),
    nn.ReLU(),
    nn.Linear(50, num_classes)
)

train_model(model1, 'model11')

  0%|          | 0/10 [00:00<?, ?it/s]

Best accuracy:

test: 0.88021
train: 0.97549


In [25]:
model2 = nn.Sequential(
    nn.BatchNorm1d(vector_size),
    nn.Linear(vector_size, 400),
    nn.BatchNorm1d(400),
    nn.ReLU(),

    nn.Linear(400, 300),
    nn.BatchNorm1d(300),
    nn.ReLU(),

    nn.Linear(300, 200),
    nn.BatchNorm1d(200),
    nn.ReLU(),

    nn.Linear(200, 100),
    nn.BatchNorm1d(100),
    nn.Dropout(p=0.3),
    nn.ReLU(),

    nn.Linear(100, 50),
    nn.BatchNorm1d(50),
    nn.Dropout(p=0.5),
    nn.ReLU(),

    nn.Linear(50, num_classes)
)

train_model(model2, 'model2')

  0%|          | 0/10 [00:00<?, ?it/s]

Best accuracy:

test: 0.85764
train: 0.96102


In [26]:
model3 = nn.Sequential(
    nn.BatchNorm1d(vector_size),
    nn.Linear(vector_size, 400),
    nn.BatchNorm1d(400),
    nn.ReLU(),

    nn.Linear(400, 300),
    nn.BatchNorm1d(300),
    nn.ReLU(),

    nn.Linear(300, 200),
    nn.BatchNorm1d(200),
    nn.ReLU(),

    nn.Linear(200, 100),
    #nn.BatchNorm1d(100),
    nn.Dropout(p=0.3),
    nn.ReLU(),

    nn.Linear(100, 50),
    #nn.BatchNorm1d(50),
    nn.Dropout(p=0.5),
    nn.ReLU(),

    nn.Linear(50, num_classes)
)

train_model(model3, 'model3')

  0%|          | 0/10 [00:00<?, ?it/s]

Best accuracy:

test: 0.87674
train: 0.9616


In [27]:
model4 = nn.Sequential(
    nn.BatchNorm1d(vector_size),
    nn.Linear(vector_size, 600),
    nn.BatchNorm1d(600),
    nn.ReLU(),

    nn.Linear(600, 400),
    nn.BatchNorm1d(400),
    nn.ReLU(),

    nn.Linear(400, 300),
    nn.BatchNorm1d(300),
    nn.ReLU(),

    nn.Linear(300, 200),
    nn.BatchNorm1d(200),
    nn.ReLU(),

    nn.Linear(200, 100),
    nn.Dropout(p=0.5),
    nn.ReLU(),

    nn.Linear(100, 50),
    nn.Dropout(p=0.5),
    nn.ReLU(),

    nn.Linear(50, num_classes)
)

train_model(model4, 'model4')

  0%|          | 0/10 [00:00<?, ?it/s]

Best accuracy:

test: 0.87153
train: 0.95832


Как можно заметить, все модели склонны к переобучению, при этом показывая хорошие результаты точности.

# Predictions
Лучшей по точности на тестовой выборке оказалась первая модель. Посчитаем F1-score для её предсказаний:

In [44]:
model1.eval()
with torch.no_grad():
    outp = model1(torch.tensor(X_test))
preds = outp.argmax(-1)

f1_score(y_test, preds)

0.8753993610223643

Теперь посторим предскзаания для test.tsv

In [46]:
test_df = pd.read_csv('/content/drive/MyDrive/skb_kontur/test.tsv', sep='\t')

In [52]:
test_df.head()

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,0
1,Ночью под Минском на президентской горе Белара...,0
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,0
3,"Сельская церковь, собравшая рекордно низкое ко...",0
4,Акции Google рухнули после объявления о переза...,0


In [62]:
test_df.shape

(1000, 2)

Получим токены тестового набора:

In [59]:
test_tokens = get_tokens(test_df)
test_tokens[:2]

[['роскомнадзор',
  'представить',
  'реестр',
  'сочетание',
  'цвет',
  ',',
  'нежелательный',
  'россия'],
 ['ночь',
  'минск',
  'президентский',
  'гора',
  'беларашмор',
  '(',
  'пик',
  'демократия',
  ')',
  'внезапно',
  'появиться',
  'лицо',
  'николай',
  'лукашенко']]

Получим усредненные эмбединги всех заголовков тестового набора:

In [63]:
test_sent_emb = get_sent_emb(test_tokens)
test_sent_emb.shape

(1000, 300)

Построим предсказание:

In [66]:
with torch.no_grad():
    output = model1(torch.tensor(test_sent_emb))
predictions = output.argmax(-1)

In [69]:
predictions.shape

torch.Size([1000])

In [71]:
test_df['is_fake'].shape

(1000,)

In [80]:
test_df['is_fake'] = predictions

In [81]:
test_df.head(10)

,title,is_fake
0,Роскомнадзор представил реестр сочетаний цвето...,1
1,Ночью под Минском на президентской горе Белара...,1
2,Бывший спичрайтер Юрия Лозы рассказал о трудно...,1
3,"Сельская церковь, собравшая рекордно низкое ко...",1
4,Акции Google рухнули после объявления о переза...,1
5,Курс доллара вырос до исторического максимума,0
6,ОПЕК назвала оптимальный уровень цен на нефть,0
7,Российская авиакомпания откроет рейсы в Тбилис...,0
8,Швейцарская горнолыжница расстреляна в доме ро...,1
9,Учреждена театральная премия имени Гарольда Пи...,0


In [82]:
test_df.to_csv('/content/drive/MyDrive/skb_kontur/predictions.tsv')